## Opponent Shooting

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
import os
import winsound

home_folder = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1'
os.chdir(home_folder)


### Functions

In [2]:
def replace_name_values2(filename):
        # replace values with dashes for compatibility
    filename = filename.replace('%','_')
    filename = filename.replace('=','_')
    filename = filename.replace('?','_')
    filename = filename.replace('&','_')
    filename = filename.replace('20Season_','')
    filename = filename.replace('_20Season','')
    filename = filename.replace('SeasonType_','')
    filename = filename.replace('sort_gdate_dir_-1_','')
    filename = filename.replace('SeasonYear_','')
    return filename

In [3]:
def get_scoring2(url_list, file_folder, option_numbers, option_names): 

        # Function scrapes data from nba.com/stats/players/shooting/ and saves to csv files 
        # Scrapes both 5-ft and by-zone shooting data

        i = 0
        optionz = np.arange(0,option_numbers, 1)

        # get first option
        for u in url_list:
            for option in optionz:
                driver.get(u)
                time.sleep(1)
                # get option xpath
                op = option + 1
                print(f'Getting {option_names[option]} for {u}...')
                print(f' option is {op}')

                try:
                        # click OPTION
                        xpath_option = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[' + str(op) + ']' 
                        elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_option)))
                except:
                        print(f'{u} did not load. Moving to next url.')
                        continue
                
                driver.find_element(by=By.XPATH, value=xpath_option).click()

                # all pages
                xpath_all = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]' 
                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_all)))
                driver.find_element(by=By.XPATH, value=xpath_all).click()

                src = driver.page_source
                parser = BeautifulSoup(src, "lxml")

                

                # if there are repreated headers in headerlist, delete them
                #headerlist = [i for n, i in enumerate(headerlist) if i not in headerlist[:n]]

                # deal with options
                
                if op == 1:
                        # Find Table
                        table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                        headers = table.findAll('th')
                        # Skip first header
                        headerlist = [h.text.strip() for h in headers[:]]  

                        # Find rows, cols, stats
                        row_names = table.findAll('a')                      
                        row_list = [b.text.strip() for b in row_names[0:]] 
                        rows = table.findAll('tr')[0:] 
                        player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]

                        cols = ['player', 'team', 'age', 'lessthan5_FGM', 'lessthan5_FGA', 'lessthan5_FG%', '5-9ft_FGM', '5-9ft_FGA', '5-9ft_FG%',
                        '10-14ft_FGM', '10-14ft_FGA', '10-14ft_FG%', '15-19ft_FGM', '15-19ft_FGA', '15-19ft_FG%', '20-24ft_FGM', '20-24ft_FGA',
                        '20-24ft_FG%', '25-29ft_FGM', '25-29ft_FGA', '25-29ft_FG%']

                        #set the length to ignore hidden columns
                        tot_cols = 21                        
                        headerlist = headerlist[:tot_cols]

                        stats = pd.DataFrame(player_stats, columns = cols)

                        filename = file_folder + str(u[34:]).replace('/', '_') + 'op_' + str(option_names[option]) + '.csv'
                        filename = replace_name_values2(filename)
                        pd.DataFrame.to_csv(stats, filename)

                        i += 1
                        lu = len(url_list)
                        print(f'{filename} Completed Successfully! {i} / {lu} Complete!')
                        
                elif op == 2: 
                        print('option 2 - do nothing')

                elif op == 3:
                        

                        u2 = u + '&DistanceRange=By+Zone'
                        driver.get(u2)
                        time.sleep(1)
                        src = driver.page_source
                        parser = BeautifulSoup(src, "lxml")

                        try:
                                # click OPTION
                                xpath_option = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[' + str(op) + ']' 
                                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_option)))
                        except:
                                print(f'{u} did not load. Moving to next url.')
                                continue
                        
                        driver.find_element(by=By.XPATH, value=xpath_option).click()

                        # all pages
                        xpath_all = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]' 
                        elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_all)))
                        driver.find_element(by=By.XPATH, value=xpath_all).click()

                        src = driver.page_source
                        parser = BeautifulSoup(src, "lxml")

                        # Find Table
                        table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                        headers = table.findAll('th')
                        # Skip first header
                        headerlist = [h.text.strip() for h in headers[:]]  

                        # Find rows, cols, stats
                        row_names = table.findAll('a')                      
                        row_list = [b.text.strip() for b in row_names[0:]] 
                        rows = table.findAll('tr')[0:] 
                        player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]

                        cols1 = ['player', 'team', 'age', 'restricted_fgm', 'restricted_fga', 'restricted_fg%', 'paint_fgm', 'paint_fga', 'paint_fg%',
                                'mid_range_fgm', 'mid_range_fga', 'mid_range_fg%', 'left_corn3_fgm', 'left_corn3_fga', 'left_corn3_fg%', 'right_corn3_fgm',
                                'right_corn3_fga', 'right_corn3_fg%', 'corner3_fgm', 'corner3_fga', 'corner3_fg%', 'above_break3_fgm', 'above_break3_fga', 'above_break3_fg%']
                        
                        #set the length to ignore hidden columns
                        tot_cols = 24                      
                        headerlist = headerlist[:tot_cols]
                        stats = pd.DataFrame(player_stats, columns = cols1)
                        filename = file_folder + str(u[34:]).replace('/', '_') + 'op_' + str(option_names[option]) + '.csv'
                        filename = replace_name_values2(filename)
                        pd.DataFrame.to_csv(stats, filename)
                        i += 1
                        lu = len(url_list)
                        print(f'{filename} Completed Successfully! {i} / {lu} Complete!')

In [4]:
def trans_urls(url):
    new_url = str(url)[34:].replace('/', '_')
    filename = replace_name_values2(new_url)
    return filename

In [5]:
def append_the_data(folder, data_prefix, filename_selector):
    # Appending data together via folder and/or file name

    path = folder
    p = os.listdir(path)
    pf = pd.DataFrame(p)


    # filter for files that contain the filename_selector
    pf_reg = pf.loc[pf[0].astype(str).str.contains(filename_selector)] 

    appended_data = []
    for file in pf_reg[0]:
        data = pd.read_csv(folder + '/' + file)
        # if "Season" a column, drop it
        if 'Season' in data.columns:
            data = data.drop(columns = ['Season'])
        
        data['season'] = file[(file.find('20')):(file.find('20'))+4]
        data['season_type'] = np.where('Regular' in file, 'Regular', 'Playoffs')
        # add prefix to columns
        data = data.add_prefix(data_prefix)
        data.columns = data.columns.str.lower()
        appended_data.append(data)
    
    appended_data = pd.concat(appended_data)
    return appended_data

In [6]:
def get_urls():
    player_opp_shooting = 'https://www.nba.com/stats/players/opponent-shooting/'
    opp_shooting_urls = []
    season_types = ['Regular+Season', 'Playoffs']
    years =['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16']

    for year in years:
        for s_types in season_types:
            url = player_opp_shooting + '?Season=' + year + '&SeasonType=' + s_types
            opp_shooting_urls.append(str(url))

    to_download = pd.DataFrame(opp_shooting_urls, columns = ['urls'])

    # new columns with filenames
    to_download['filename1'] = to_download.apply(lambda row: trans_urls(row['urls']), axis=1)
    to_download['filename_o1'] = to_download['filename1'] + 'op_5ft_range.csv'
    to_download['filename_o2'] = to_download['filename1'] + 'op_by_zone.csv'

    return to_download

In [7]:
to_download = get_urls()

In [8]:
# Check to see if files already exist
folder1 = 'data/player/shot_dashboard/opponent_shooting/regular_season'
f1_files = os.listdir(folder1)

folder2 = 'data/player/shot_dashboard/opponent_shooting/playoffs'
f2_files = os.listdir(folder2)

files = f1_files + f2_files
files = [f for f in files if '.csv' in f]

# Filter out files that already exist
to_download = to_download[~to_download['filename_o1'].isin(files)]
to_download = to_download[~to_download['filename_o2'].isin(files)]

to_download


,urls,filename1,filename_o1,filename_o2


In [9]:
if to_download.shape[0] > 0:
    # Download the files
    driver = webdriver.Chrome()
    get_scoring2(to_download['urls'], 'data/player/shot_dashboard/opponent_shooting/', 3, ['5ft_range', '8ft_range', 'by_zone'])

    # move files to correct folder
    op_shooting_files = os.listdir('data/player/shot_dashboard/opponent_shooting/')
    op_shooting_files = [f for f in op_shooting_files if '.csv' in f]
    for file in op_shooting_files:
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/opponent_shooting/' + file, 'data/player/shot_dashboard/opponent_shooting/regular_season/' + file)
        else:
            shutil.move('data/player/shot_dashboard/opponent_shooting/' + file, 'data/player/shot_dashboard/opponent_shooting/playoffs/' + file)

else:
    print('No new files to download')

No new files to download


### Update This Year

In [12]:
# get list of this years urls
player_opp_shooting = 'https://www.nba.com/stats/players/opponent-shooting/?Season=2022-23&SeasonType=Regular+Season'
driver = webdriver.Chrome()
get_scoring2([player_opp_shooting], 'data/player/shot_dashboard/opponent_shooting/', 3, ['5ft_range', '8ft_range', 'by_zone'])

Getting 5ft_range for https://www.nba.com/stats/players/opponent-shooting/?Season=2022-23&SeasonType=Regular+Season...
 option is 1
data/player/shot_dashboard/opponent_shooting/opponent-shooting__Season_2022-23_Regular+Seasonop_5ft_range.csv Completed Successfully! 1 / 1 Complete!
Getting 8ft_range for https://www.nba.com/stats/players/opponent-shooting/?Season=2022-23&SeasonType=Regular+Season...
 option is 2
option 2 - do nothing
Getting by_zone for https://www.nba.com/stats/players/opponent-shooting/?Season=2022-23&SeasonType=Regular+Season...
 option is 3
data/player/shot_dashboard/opponent_shooting/opponent-shooting__Season_2022-23_Regular+Seasonop_by_zone.csv Completed Successfully! 2 / 1 Complete!


In [13]:
# move the files to the correct folder
op_shooting_files = os.listdir('data/player/shot_dashboard/opponent_shooting/')
op_shooting_files = [f for f in op_shooting_files if '.csv' in f]
for file in op_shooting_files:
    if 'Regular' in file:
        shutil.move('data/player/shot_dashboard/opponent_shooting/' + file, 'data/player/shot_dashboard/opponent_shooting/regular_season/' + file)
    else:
        shutil.move('data/player/shot_dashboard/opponent_shooting/' + file, 'data/player/shot_dashboard/opponent_shooting/playoffs/' + file)

## Append

In [14]:
opp_shooting_reg1 = append_the_data('data/player/shot_dashboard/opponent_shooting/regular_season', 'opp_shooting_5ft_', '5ft')
opp_shooting_reg2 = append_the_data('data/player/shot_dashboard/opponent_shooting/regular_season', 'opp_shooting_byzone_', 'by_zone')

opp_shooting_play1 = append_the_data('data/player/shot_dashboard/opponent_shooting/playoffs', 'opp_shooting_5ft_', '5ft')
opp_shooting_play2 = append_the_data('data/player/shot_dashboard/opponent_shooting/playoffs', 'opp_shooting_byzone_', 'by_zone')


In [15]:
opp_shooting_reg1

,opp_shooting_5ft_unnamed: 0,opp_shooting_5ft_player,opp_shooting_5ft_team,opp_shooting_5ft_age,opp_shooting_5ft_lessthan5_fgm,opp_shooting_5ft_lessthan5_fga,opp_shooting_5ft_lessthan5_fg%,opp_shooting_5ft_5-9ft_fgm,opp_shooting_5ft_5-9ft_fga,opp_shooting_5ft_5-9ft_fg%,...,opp_shooting_5ft_15-19ft_fga,opp_shooting_5ft_15-19ft_fg%,opp_shooting_5ft_20-24ft_fgm,opp_shooting_5ft_20-24ft_fga,opp_shooting_5ft_20-24ft_fg%,opp_shooting_5ft_25-29ft_fgm,opp_shooting_5ft_25-29ft_fga,opp_shooting_5ft_25-29ft_fg%,opp_shooting_5ft_season,opp_shooting_5ft_season_type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,Regular
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,Regular
2,2,Aaron Brooks,CHI,31.0,5.8,9.7,59.7,1.1,2.9,36.5,...,4.4,37.5,2.1,5.9,35.0,1.1,3.5,32.9,2015,Regular
3,3,Aaron Gordon,ORL,20.0,9.1,15.7,57.9,1.5,3.9,38.4,...,5.1,38.7,2.9,7.6,38.6,2.2,5.6,39.3,2015,Regular
4,4,Aaron Harrison,CHA,21.0,1.7,2.8,61.0,0.3,0.8,33.3,...,1.4,30.4,0.4,1.4,32.0,0.6,1.6,38.7,2015,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,486,Zach Collins,SAS,25.0,9.1,15.2,59.7,1.6,4.1,39.5,...,2.9,54.1,2.0,4.9,41.7,3.1,9.5,33.2,2022,Regular
487,487,Zach LaVine,CHI,27.0,13.6,20.5,66.7,2.6,6.3,41.7,...,3.7,39.4,4.1,10.3,39.4,6.1,15.9,38.6,2022,Regular
488,488,Zeke Nnaji,DEN,21.0,3.4,5.2,65.5,1.4,2.5,56.0,...,0.8,37.5,1.0,2.8,35.7,1.2,4.4,27.4,2022,Regular
489,489,Ziaire Williams,MEM,21.0,5.4,10.6,50.6,0.5,2.6,19.0,...,2.0,56.3,2.0,6.6,30.2,3.1,10.4,30.1,2022,Regular


In [16]:
opp_shooting_reg = pd.merge(opp_shooting_reg1, opp_shooting_reg2,
                        left_on = ['opp_shooting_5ft_player', 'opp_shooting_5ft_team', 'opp_shooting_5ft_season','opp_shooting_5ft_season_type'],
                        right_on = ['opp_shooting_byzone_player', 'opp_shooting_byzone_team', 'opp_shooting_byzone_season','opp_shooting_byzone_season_type'],
                        how = 'left')

opp_shooting_play = pd.merge(opp_shooting_play1, opp_shooting_play2,
                        left_on = ['opp_shooting_5ft_player', 'opp_shooting_5ft_team', 'opp_shooting_5ft_season','opp_shooting_5ft_season_type'],
                        right_on = ['opp_shooting_byzone_player', 'opp_shooting_byzone_team', 'opp_shooting_byzone_season','opp_shooting_byzone_season_type'],
                        how = 'left')



In [17]:
all_opponent_shooting = pd.concat([opp_shooting_reg, opp_shooting_play])
all_opponent_shooting.to_csv('data/player/aggregates/All_Opponent_Shooting.csv', index = False)